In [1]:
import chess.pgn
import pandas as pd
from sqlalchemy import create_engine, text
import os
from datetime import datetime

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [2]:
#Database configuration
DB_USER = os.getenv('USER')
DB_NAME = 'chess_app'
DATABASE_URL = f'postgresql://{DB_USER}:@localhost:5432/{DB_NAME}'

#Create Engine
engine = create_engine(DATABASE_URL, echo=False)

#Test Connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM games;"))
    count = result.fetchone()[0]
    print(f"✅ Connected to database!")
    print(f"Current games in database: {count}")

✅ Connected to database!
Current games in database: 100


In [3]:
def parse_pgn_file(pgn_file_path, max_games=100):
    """
    Parse PGN file and extract game data
    
    Args:
        pgn_file_path: Path to PGN file
        max_games: Maximum number of games to parse
    
    Returns:
        DataFrame with game data
    """
    games_data = []
    
    with open(pgn_file_path) as pgn:
        game_count = 0
        
        while game_count < max_games:
            game = chess.pgn.read_game(pgn)
            
            if game is None:
                break
            
            headers = game.headers
            
            # Extract game data
            game_data = {
                'white_player': headers.get('White', 'Unknown'),
                'black_player': headers.get('Black', 'Unknown'),
                'white_elo': int(headers.get('WhiteElo', 0)) if headers.get('WhiteElo', '').isdigit() else None,
                'black_elo': int(headers.get('BlackElo', 0)) if headers.get('BlackElo', '').isdigit() else None,
                'result': headers.get('Result', '*'),
                'game_date': headers.get('Date', '').replace('.', '-') if headers.get('Date') else None,
                'opening_name': headers.get('Opening', 'Unknown'),
                'pgn_moves': game.board().variation_san(game.mainline_moves()),
                'source': 'lichess'
            }
            
            games_data.append(game_data)
            game_count += 1
            
            # Progress indicator
            if game_count % 10 == 0:
                print(f"Parsed {game_count} games...", end='\r')
    
    print(f"\n✅ Parsed {len(games_data)} games total!")
    return pd.DataFrame(games_data)

print("✅ Parse function defined!")

✅ Parse function defined!


In [4]:
import os

# Get the absolute path to your project root
project_root = os.path.expanduser('~/chess_project')

# Path to your sample PGN file (absolute path)
pgn_path = os.path.join(project_root, 'data/raw/sample_1000_games.pgn')

# Verify the file exists
if os.path.exists(pgn_path):
    print(f"✅ Found file: {pgn_path}")
    file_size = os.path.getsize(pgn_path) / (1024 * 1024)  # Convert to MB
    print(f"File size: {file_size:.2f} MB")
else:
    print(f"❌ File not found: {pgn_path}")
    print("\nLet's check what's in data/raw/:")
    raw_dir = os.path.join(project_root, 'data/raw')
    if os.path.exists(raw_dir):
        print(os.listdir(raw_dir))
    else:
        print("data/raw directory doesn't exist!")

# Parse 100 games first (to test)
print("\nStarting to parse PGN file...")
games_df = parse_pgn_file(pgn_path, max_games=100)

# Display first few games
print("\nFirst 5 games:")
games_df.head()

✅ Found file: /Users/miguel/chess_project/data/raw/sample_1000_games.pgn
File size: 2.23 MB

Starting to parse PGN file...
Parsed 100 games...
✅ Parsed 100 games total!

First 5 games:


,white_player,black_player,white_elo,black_elo,result,game_date,opening_name,pgn_moves,source
0,Detectie,youssefaymn,1176,1155,1-0,2024-01-01,Italian Game: Anti-Fried Liver Defense,1. e4 e5 2. Nf3 Nc6 3. Bc4 h6 4. O-O Nf6 5. d3...,lichess
1,LionsAK32,Vusal_1990,732,763,0-1,2024-01-01,Vienna Game: Stanley Variation,1. e4 e5 2. Bc4 Nf6 3. Nc3 d6 4. Qf3 Be7 5. d4...,lichess
2,DRAGONKING54,fanydacute,1783,1784,1-0,2024-01-01,Dutch Defense: Fianchetto Attack,1. d4 f5 2. g3 Nf6 3. Bg2 e6 4. Nf3 c5 5. c3 N...,lichess
3,TRUFFE68,Davigoal,1491,1495,0-1,2024-01-01,Polish Opening,1. b4 e6 2. Bb2 b6 3. Nf3 Bb7 4. g3 f6 5. c3 N...,lichess
4,rodrigorramos,ThomasTuchel,1391,1381,0-1,2024-01-01,Modern Defense: Standard Line,1. d4 g6 2. e4 Bg7 3. Nc3 Na6,lichess


In [5]:
# Basic statistics
print("=" * 60)
print("DATASET STATISTICS")
print("=" * 60)

print(f"\n📊 Total games parsed: {len(games_df)}")

print(f"\n🏆 Results distribution:")
print(games_df['result'].value_counts())

print(f"\n⭐ ELO Ratings:")
print(f"   White average: {games_df['white_elo'].mean():.0f}")
print(f"   Black average: {games_df['black_elo'].mean():.0f}")
print(f"   Highest rated: {games_df[['white_elo', 'black_elo']].max().max()}")
print(f"   Lowest rated: {games_df[['white_elo', 'black_elo']].min().min()}")

print(f"\n♟️  Top 10 Most Common Openings:")
print(games_df['opening_name'].value_counts().head(10))

print(f"\n📅 Date range:")
print(f"   From: {games_df['game_date'].min()}")
print(f"   To: {games_df['game_date'].max()}")

DATASET STATISTICS

📊 Total games parsed: 100

🏆 Results distribution:
result
1-0        51
0-1        47
1/2-1/2     2
Name: count, dtype: int64

⭐ ELO Ratings:
   White average: 1655
   Black average: 1659
   Highest rated: 2658
   Lowest rated: 635

♟️  Top 10 Most Common Openings:
opening_name
Van't Kruijs Opening                            5
Horwitz Defense                                 4
Queen's Pawn Game: Accelerated London System    3
Modern Defense                                  3
Queen's Pawn Game                               3
Italian Game: Anti-Fried Liver Defense          2
Sicilian Defense: Franco-Sicilian Variation     2
Indian Defense                                  2
Owen Defense                                    2
Sicilian Defense: Modern Variations             2
Name: count, dtype: int64

📅 Date range:
   From: 2024-01-01
   To: 2024-01-01


In [6]:
# Load games into database
print("=" * 60)
print("LOADING DATA INTO POSTGRESQL")
print("=" * 60)

try:
    # Insert games into database
    games_df.to_sql('games', engine, if_exists='append', index=False)
    
    print(f"\n✅ Successfully loaded {len(games_df)} games into database!")
    
    # Verify
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM games;"))
        total = result.fetchone()[0]
        print(f"📊 Total games now in database: {total}")
        
except Exception as e:
    print(f"❌ Error: {e}")

LOADING DATA INTO POSTGRESQL

✅ Successfully loaded 100 games into database!
📊 Total games now in database: 200


In [7]:
# Query games from the database
print("=" * 60)
print("QUERYING DATABASE")
print("=" * 60)

query = """
SELECT 
    game_id,
    white_player,
    black_player,
    white_elo,
    black_elo,
    result,
    opening_name
FROM games
ORDER BY (white_elo + black_elo) DESC
LIMIT 10;
"""

print("\n🏆 Top 10 Highest Rated Games:\n")
result_df = pd.read_sql(query, engine)
result_df

QUERYING DATABASE

🏆 Top 10 Highest Rated Games:



,game_id,white_player,black_player,white_elo,black_elo,result,opening_name
0,183,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
1,83,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
2,185,MeMoNiToS,Dagoberto_und_Ich,2446,2471,0-1,Nimzo-Larsen Attack: Classical Variation
3,85,MeMoNiToS,Dagoberto_und_Ich,2446,2471,0-1,Nimzo-Larsen Attack: Classical Variation
4,6,Peixeiro,VaRYemezAmca72,2504,2371,1-0,"Réti Opening: Anglo-Slav Variation, Bogoljubow..."
5,106,Peixeiro,VaRYemezAmca72,2504,2371,1-0,"Réti Opening: Anglo-Slav Variation, Bogoljubow..."
6,80,goosesome,Kastigador,2394,2420,1-0,King's Indian Attack
7,180,goosesome,Kastigador,2394,2420,1-0,King's Indian Attack
8,84,inner_join,andrescaballero34,2341,2336,1-0,"Sicilian Defense: Classical Variation, Sozin A..."
9,184,inner_join,andrescaballero34,2341,2336,1-0,"Sicilian Defense: Classical Variation, Sozin A..."


In [8]:
# Query games from the database
print("=" * 60)
print("QUERYING DATABASE")
print("=" * 60)

query = """
SELECT 
    game_id,
    white_player,
    black_player,
    white_elo,
    black_elo,
    result,
    opening_name
FROM games
ORDER BY (white_elo + black_elo) DESC
LIMIT 10;
"""

print("\n🏆 Top 10 Highest Rated Games:\n")
result_df = pd.read_sql(query, engine)
result_df

QUERYING DATABASE

🏆 Top 10 Highest Rated Games:



,game_id,white_player,black_player,white_elo,black_elo,result,opening_name
0,183,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
1,83,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
2,185,MeMoNiToS,Dagoberto_und_Ich,2446,2471,0-1,Nimzo-Larsen Attack: Classical Variation
3,85,MeMoNiToS,Dagoberto_und_Ich,2446,2471,0-1,Nimzo-Larsen Attack: Classical Variation
4,6,Peixeiro,VaRYemezAmca72,2504,2371,1-0,"Réti Opening: Anglo-Slav Variation, Bogoljubow..."
5,106,Peixeiro,VaRYemezAmca72,2504,2371,1-0,"Réti Opening: Anglo-Slav Variation, Bogoljubow..."
6,80,goosesome,Kastigador,2394,2420,1-0,King's Indian Attack
7,180,goosesome,Kastigador,2394,2420,1-0,King's Indian Attack
8,84,inner_join,andrescaballero34,2341,2336,1-0,"Sicilian Defense: Classical Variation, Sozin A..."
9,184,inner_join,andrescaballero34,2341,2336,1-0,"Sicilian Defense: Classical Variation, Sozin A..."


In [9]:
def extract_positions_from_game(game_id, pgn_moves_str):
    """
    Extract individual positions from a game
    
    Args:
        game_id: The game ID from database
        pgn_moves_str: String of moves like "1. e4 e5 2. Nf3 Nc6..."
    
    Returns:
        List of position dictionaries
    """
    import io
    
    positions = []
    
    try:
        # Create a game from the moves
        game = chess.pgn.read_game(io.StringIO(f"[Event \"\"]\n\n{pgn_moves_str}"))
        
        if game is None:
            return positions
        
        board = game.board()
        move_number = 0
        
        # Iterate through all moves
        for move in game.mainline_moves():
            move_number += 1
            
            # Record position BEFORE the move
            fen = board.fen()
            move_san = board.san(move)
            
            position_data = {
                'game_id': game_id,
                'move_number': move_number,
                'fen': fen,
                'move_played': move_san
            }
            
            positions.append(position_data)
            
            # Make the move
            board.push(move)
    
    except Exception as e:
        print(f"Error processing game {game_id}: {e}")
    
    return positions

print("✅ Position extraction function defined!")

✅ Position extraction function defined!


In [10]:
print("=" * 60)
print("EXTRACTING POSITIONS FROM GAMES")
print("=" * 60)

# Get first 10 games from database
query = """
SELECT game_id, pgn_moves
FROM games
LIMIT 10;
"""

games_to_process = pd.read_sql(query, engine)

all_positions = []

for idx, row in games_to_process.iterrows():
    game_id = row['game_id']
    pgn_moves = row['pgn_moves']
    
    positions = extract_positions_from_game(game_id, pgn_moves)
    all_positions.extend(positions)
    
    print(f"Game {game_id}: extracted {len(positions)} positions", end='\r')

print(f"\n\n✅ Total positions extracted: {len(all_positions)}")

# Convert to DataFrame
positions_df = pd.DataFrame(all_positions)

print("\nFirst 5 positions:")
positions_df.head()

EXTRACTING POSITIONS FROM GAMES
Game 10: extracted 114 positions

✅ Total positions extracted: 673

First 5 positions:


,game_id,move_number,fen,move_played
0,1,1,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,e4
1,1,2,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,e5
2,1,3,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,Nf3
3,1,4,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQK...,Nc6
4,1,5,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,Bc4


In [11]:
print("=" * 60)
print("LOADING POSITIONS INTO DATABASE")
print("=" * 60)

try:
    # Insert positions into database
    positions_df.to_sql('positions', engine, if_exists='append', index=False)
    
    print(f"\n✅ Successfully loaded {len(positions_df)} positions into database!")
    
    # Verify
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM positions;"))
        total = result.fetchone()[0]
        print(f"📊 Total positions now in database: {total}")
        
except Exception as e:
    print(f"❌ Error: {e}")

LOADING POSITIONS INTO DATABASE

✅ Successfully loaded 673 positions into database!
📊 Total positions now in database: 673


In [12]:
# Query some positions
query = """
SELECT 
    p.position_id,
    p.game_id,
    p.move_number,
    p.move_played,
    p.fen,
    g.white_player,
    g.black_player
FROM positions p
JOIN games g ON p.game_id = g.game_id
LIMIT 10;
"""

print("Sample positions from database:\n")
pd.read_sql(query, engine)

Sample positions from database:



,position_id,game_id,move_number,move_played,fen,white_player,black_player
0,1,1,1,e4,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,Detectie,youssefaymn
1,2,1,2,e5,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,Detectie,youssefaymn
2,3,1,3,Nf3,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,Detectie,youssefaymn
3,4,1,4,Nc6,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQK...,Detectie,youssefaymn
4,5,1,5,Bc4,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,Detectie,youssefaymn
5,6,1,6,h6,r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/R...,Detectie,youssefaymn
6,7,1,7,O-O,r1bqkbnr/pppp1pp1/2n4p/4p3/2B1P3/5N2/PPPP1PPP/...,Detectie,youssefaymn
7,8,1,8,Nf6,r1bqkbnr/pppp1pp1/2n4p/4p3/2B1P3/5N2/PPPP1PPP/...,Detectie,youssefaymn
8,9,1,9,d3,r1bqkb1r/pppp1pp1/2n2n1p/4p3/2B1P3/5N2/PPPP1PP...,Detectie,youssefaymn
9,10,1,10,d6,r1bqkb1r/pppp1pp1/2n2n1p/4p3/2B1P3/3P1N2/PPP2P...,Detectie,youssefaymn
